# Import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Import data bases

In [ ]:
df = pd.read_csv('../data/base_modelis_pass_det.csv',sep = ';')

In [ ]:
df.shape

## analyse des types de var

In [ ]:
df.select_dtypes(include = 'object').shape[1]

In [ ]:
df.select_dtypes(include = 'float').shape[1]

In [ ]:
df.select_dtypes(include = 'int').shape[1]

# Valeurs manquantes

In [ ]:
df_temp = df.drop_duplicates(subset = 'cd_op', keep = 'last')

In [ ]:
df_temp.shape

In [ ]:
df_temp.isna().sum()/df_temp.shape[0]>0.4

# To do list

Analyse exploratoire : distrib var quali/quanti, visualisation NaN, corrélations, doublons, outliers

Analyse + profonde : que prédire, quelles vars sont pertinentes, diff entre les 2 df, liste questions à poser

Axel : distrib quanti, correlations, outliers

Baheux : distrib quali ,NaN , doublons

### Interrogations :
Faut-il ne garder que la dernière ligne de chaque individu, correspondant à sa sortie de defaut ou bien à son DET pour faire un logit? Ou bien doit on garder toutes les observations au risque d'avoir trop peu de 0 et donc de moins bien classifier?

Penser au problème de troncature/censure des données (faut il supprimer les individus dont le dossier est encore en cours?)

### Conversion des dates en datetime

In [ ]:
df['date_entree_defaut'] = pd.to_datetime(df['date_entree_defaut'])

df['date_sortie_defaut'] = pd.to_datetime(df['date_sortie_defaut'])

df['arrete'] = pd.to_datetime(df['arrete'])

df['dt_arr_last_enc_ope'] = pd.to_datetime(df['dt_arr_last_enc_ope'])
df['dt_arr_1st_enc_ope'] = pd.to_datetime(df['dt_arr_1st_enc_ope'])

df['DT_MEP_OPE'] = pd.to_datetime(df['DT_MEP_OPE'], format='%d%b%Y:%H:%M:%S.%f')


df['dtHJD_prov'] = pd.to_datetime(df['dtHJD_prov'])
df['dtHJD_def'] = pd.to_datetime(df['dtHJD_def'])

df['date_sortie_defaut'] = pd.to_datetime(df['date_sortie_defaut'])

# Analyse préliminaire

In [ ]:
df_obj = df.drop(columns = ['CD_NAT_EMP2','sol_echec']).select_dtypes(include='object').drop(columns = 'cd_op').copy()


In [ ]:
for col in df_obj.columns:
    print(f"{col} : {df_obj[col].nunique()}")

In [ ]:
df.drop_duplicates(subset = 'cd_op', keep = 'last')['fl_pass_DET'].value_counts()

In [ ]:
df.head(50)

- On a 23 305 individus sur l'ensemble de la période
- 7463 d'entre eux sont passés en DET
- Le reste a remboursé le défaut


# Analyse var quali sur dataframe total

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# Définir le nombre de graphiques par ligne
graphs_per_row = 3

# Calculer le nombre total de lignes nécessaires
num_rows = -(-len(df_obj.columns) // graphs_per_row)  # Utilise la division entière vers le haut

# Créer une grille de sous-graphiques
fig = make_subplots(rows=num_rows, cols=graphs_per_row, subplot_titles=df_obj.columns)

for i, col in enumerate(df_obj.columns):
    category_counts = df_obj[col].value_counts().reset_index()
    category_counts.columns = ['Catégorie', 'Fréquence']

    # Création du diagramme à barres avec Plotly Express
    bar_fig = px.bar(category_counts, x='Catégorie', y='Fréquence', title=col,
                     labels={'Catégorie': 'Catégorie', 'Fréquence': 'Fréquence'})

    # Ajout du graphique à la grille
    row = i // graphs_per_row + 1
    col = i % graphs_per_row + 1
    fig.add_trace(bar_fig.data[0], row=row, col=col)

# Mettre à jour la mise en page
fig.update_layout(height=num_rows * 200, width=graphs_per_row * 400, title_text="Graphiques par colonne")
fig.show()


# Distributions des var quanti

In [ ]:
from plotly.subplots import make_subplots

# Définir le nombre de graphiques par ligne
graphs_per_row = 3

df_num = df.drop(columns = ['NB_RP_HOP', 'mt_inscrip_devant_CL_afr', 'valeur_bien_afr',
'CD_CSP_EMP2', 'LTV_actu', 'NB_LO_HOP', 
'LTVnonactu_1an_avant_def', 'NB_RS_HOP',
'dispo_HP_afr', 'CD_NAT_EMP2', 'dtHJD_prov',
'dtHJD_def', 'dtSRT1st', 'AGE_EMP2', 'CD_SITFAM_EMP2'])
float_columns = df_num.select_dtypes(include='float').columns

num_cols = 3
num_rows = (len(float_columns) + 1) // 2  # Ajoutez 1 pour gérer les cas impairs

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 4 * num_rows))

fig.subplots_adjust(hspace=0.5)

axes = axes.flatten()

for i, col in enumerate(float_columns):
    sns.kdeplot(x=df[col], ax=axes[i], fill=True)
    axes[i].set_title(f'Kernel Plot de {col}')

plt.show()

# Stabilité des catégories dans le tps par rapport à la target 

In [ ]:
cat_columns = ['CD_NAT_EMP1','CD_MTF_DFC']

In [ ]:
import matplotlib.pyplot as plt

for col in df.select_dtypes(include = 'object').columns:
    # Calcul de la moyenne par groupe (genre et date)
    result = df.groupby([col, 'arrete'])['fl_pass_DET'].mean().reset_index()
    
    # Tracé des données
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for genre, group in result.groupby(f'{col}'):
        ax.plot(group['arrete'], group['fl_pass_DET'], label=f'{genre}')
    
    # Ajout de légendes et labels
    ax.legend()
    ax.set_xlabel('Date')
    ax.set_ylabel('Moyenne de la target binaire')
    ax.set_title(f'Évolution de la moyenne de la target par modalité de {col} dans le temps')
    
    # Affichage du graphique
    plt.show()


### Dates

In [ ]:
df.select_dtypes(include='datetime').columns

In [ ]:
for col in df.select_dtypes(include = 'datetime').columns:
    fig = px.histogram(df[col], title=col,
                 labels={'Catégorie': 'Catégorie', 'Fréquence': 'Fréquence'})

    # Affichage du diagramme
    fig.show()

# Analyse dépendances temporelles

In [ ]:
var_to_categorize = [
    "fl_pass_DET",
    "CD_POST_BIEN_PFI",
    "FL_REL_NON_ANNULE",
    "FL_ETR",
    "CD_ETAT_CIVIL",
    "CD_DNE_CHR_LGD_SAIN",
    "fl_prise_de_gar",
    "fl_fam_vam",
    "fam_ETH",
    "fam_ENQ",
    "fam_exp",
    "fam_hyp",
    "fam_sim",
    "fam_AEP",
    "dat_dec_echec",
    "solution",
    "fam_PCD",
    "CD_MTF_ENE_CTX",
    "niv_gravite_mtf_dne",
    "niv_gravite_mtf_dfs",
    "no_pass_def",
    "fl_def_ss_pass_CTX",
    "fl_prt_Conso",
    "fl_fonc",
    "AMI",
    "EXE",
    "PTG"
]

for var in var_to_categorize : 
    df[var] = df[var].astype("object")

In [ ]:
liste= []
for col in df.select_dtypes(include = 'number').columns:
    if df[col].nunique()>20:
        liste.append(col)

In [ ]:
import matplotlib.pyplot as plt

grouped_df = df.iloc[:100,:].groupby('cd_op')

for cd_op, group in grouped_df:
    
    plt.figure()
    
    for col in group[liste].columns:
        plt.plot(group['arrete'], group[col], label=col)
    
    plt.xlabel('arrete')
    plt.ylabel('Valeur')
    plt.title(f'Evolution des variables pour cd_op={cd_op}')
    plt.legend('off')
    
    plt.show()